In [2]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [3]:
new_directory = "C:\TradeGroup"
os.chdir(new_directory)
my_wd = new_directory

In [4]:
def generate_HS2(tcode):
    tcode=str(tcode)
    if len(tcode)==7 or len(tcode)==5 or len(tcode)==9:
        return tcode[0]
    elif len(tcode)==8 or len(tcode)==6 or len(tcode)==10:
        return tcode[:2]

In [21]:
baseyear=1380
for year in range(1380,1403):
    exports_file_path = f"{my_wd}/Data/Cleaned_Data/Y{year}Exports.xlsx"
    imports_file_path = f"{my_wd}/Data/Cleaned_Data/Y{year}Imports.xlsx"
    #if year!=1394:
    Exports = pd.read_excel(exports_file_path)
    Imports = pd.read_excel(imports_file_path)

    Imports.dropna(subset='Tcode',inplace=True)
    Exports.dropna(subset="Tcode",inplace=True)

    Imports['HS2']=Imports['Tcode'].astype('int64').apply(generate_HS2)
    Exports['HS2']=Exports['Tcode'].astype('int64').apply(generate_HS2)

    Imports.replace("ي",'ی',inplace=True)
    Imp_China=Imports[Imports['Country']=="چين"]

    Exports.replace("ي",'ی',inplace=True)
    Exp_China=Exports[Exports['Country']=="چين"]

    Imp_China_HS2=Imp_China.groupby('HS2').agg(
        Dollars = ('Dollars','sum')
        ).reset_index()
    


    Exp_China_HS2=Exp_China.groupby('HS2').agg(
        Dollars = ('Dollars','sum')
        ).reset_index() 

    Imp_China_HS2.rename(columns={"Dollars": f'Imp_{year}_D'},inplace=True)
    Exp_China_HS2.rename(columns={"Dollars": f'Exp_{year}_D'},inplace=True)

    if year == baseyear:
        Imp_China_HS2_Series=Imp_China_HS2
        Exp_China_HS2_Series=Exp_China_HS2
    else:
        Imp_China_HS2_Series=Imp_China_HS2_Series.merge(Imp_China_HS2,on='HS2',how='outer')
        Exp_China_HS2_Series=Exp_China_HS2_Series.merge(Exp_China_HS2,on='HS2',how='outer')       
       
   


In [26]:
Imp_China_HS2_Series.fillna(0,inplace=True)
Exp_China_HS2_Series.fillna(0,inplace=True)

Exp_China_HS2_Series.to_excel(f'{my_wd}\Data\Indices\IranExpChina.xlsx',index=False)
Imp_China_HS2_Series.to_excel(f'{my_wd}\Data\Indices\IranImpChina.xlsx',index=False)


